In [1]:
import json
from pathlib import Path

from datasets import load_dataset
from transformers import AutoTokenizer, BertForSequenceClassification, DataCollatorWithPadding, Trainer, TrainingArguments, pipeline
from peft import LoraConfig, TaskType, get_peft_model
import numpy as np
import torch

/home/lukasz/miniconda3/envs/nlp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
json_files = list(Path("./data/ground_truth").rglob("*.jsonl"))

In [3]:
full_dataset = []

for file in json_files:
    with open(file, "r") as f:
        lines = [json.loads(line) for line in f]
    full_dataset.extend(lines)

In [4]:
mapping = {
    "Mowa nienawiści": "Hate",
    "Neutralny": "Neutral",
    "Odwracanie": "Neutral",
    "Wzmacnianie": "Hate"
}

def get_label(labels):
    label = labels[0][2]
    if label in mapping:
        label = mapping[label]
    return label

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
data_files = {"train": "train_with_masks.jsonl", "test": "test.jsonl"}
dataset = load_dataset("./sentences", data_files=data_files)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1078
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 60
    })
})


In [7]:
sorted_labels = sorted(set([sample["label"] for sample in dataset["train"]]))
label2id = dict(zip(sorted_labels, range(0, len(sorted_labels))))
id2label = dict(zip(range(0, len(sorted_labels)), sorted_labels))

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased", return_tensors="pt")
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased",
                                                                      num_labels=len(label2id),
                                                                      label2id=label2id,
                                                                      id2label=id2label)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def preprocess_function(examples):
    """Preprocess the dataset by returning tokenized examples."""
    tokens = tokenizer(examples["text"], padding="max_length", truncation=True)
    tokens['label'] = [label2id[l] for l in examples["label"]]
    return tokens

splits = ['train', 'test']

tokenized_ds = {}

for split in splits:
    tokenized_ds[split] = dataset[split].map(preprocess_function, batched=True)

print(tokenized_ds)

Map: 100%|██████████| 60/60 [00:00<00:00, 6447.82 examples/s]

{'train': Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1078
}), 'test': Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 60
})}


In [9]:
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=64, lora_alpha=1, lora_dropout=0.1
)

peft_model = get_peft_model(model, lora_config)
print(peft_model.bert)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): lora.Linear(
              (base_layer): Linear(in_features=768, out_features=768, bias=True)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.1, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=768, out_features=64, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=64, out_features=768, bias=False)
              )
              (lora_embedding_

In [10]:
from sklearn.metrics import f1_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    f1 = f1_score(labels, predictions) 
    return {"accuracy": (predictions == labels).mean()*100, "f1": f1}


trainer = Trainer(
    model=peft_model,
    args=TrainingArguments(
        output_dir="bert-sentence-clf",
        learning_rate=2e-3,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=15,
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    processing_class=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

print("Starting to train...")
trainer.train()

Starting to train...


                                                  
  7%|▋         | 270/4050 [00:52<10:32,  5.97it/s]

{'eval_loss': 0.4134162366390228, 'eval_accuracy': 86.66666666666667, 'eval_f1': 0.9285714285714286, 'eval_runtime': 1.1902, 'eval_samples_per_second': 50.412, 'eval_steps_per_second': 12.603, 'epoch': 1.0}


 12%|█▏        | 501/4050 [01:36<10:49,  5.47it/s]

{'loss': 0.4447, 'grad_norm': 0.8074259757995605, 'learning_rate': 0.0017530864197530865, 'epoch': 1.85}


...

                                                   
100%|██████████| 4050/4050 [13:23<00:00,  5.78it/s]

{'eval_loss': 0.4737564027309418, 'eval_accuracy': 91.66666666666666, 'eval_f1': 0.9514563106796117, 'eval_runtime': 1.1497, 'eval_samples_per_second': 52.186, 'eval_steps_per_second': 13.047, 'epoch': 15.0}


100%|██████████| 4050/4050 [13:24<00:00,  5.04it/s]

{'train_runtime': 804.1812, 'train_samples_per_second': 20.107, 'train_steps_per_second': 5.036, 'train_loss': 0.24907605191807689, 'epoch': 15.0}


TrainOutput(global_step=4050, training_loss=0.24907605191807689, metrics={'train_runtime': 804.1812, 'train_samples_per_second': 20.107, 'train_steps_per_second': 5.036, 'total_flos': 4371778399887360.0, 'train_loss': 0.24907605191807689, 'epoch': 15.0})

In [11]:
from sklearn.manifold import TSNE
import pandas as pd
import plotly.express as px

In [13]:
test_ds = dataset["test"]
texts = [sample["text"] for sample in test_ds] 
inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)

bert = peft_model.base_model.model.bert

bert = bert.cpu()
bert.eval()
with torch.no_grad():
    outputs = bert(**inputs)

tsne = TSNE()
to_vis = tsne.fit_transform(outputs.last_hidden_state[:,0,:].squeeze().numpy())

df = pd.DataFrame(to_vis)
df["label"] = [post["label"] for post in test_ds]
df["text"] = [post["text"] for post in test_ds]
fig = px.scatter(df, x=0, y=1, hover_data="text", color="label")
fig.show()